## 트리오 예측 운영 프로세스

### 1. 마감 시간 직후 크롤러 실행
    - 크롤러 종료되면 게시글 히든
    


### 2. 트리오 예측 당일 점수 계산
    - 댓글 크롤링 결과를 '트리오 예측 점수계산_(오늘날짜).xlsx' 파일의 '오늘 유저 점수계산' 시트에 붙여넣음
    - 데이터 탭의 '중복된 항목 제거' 기능을 이용하여 중복 닉네임 제거 (중복 참가 및 점수 중복 누적 방지)
    - '오늘 선수별 점수계산' 시트 계산결과가 잘 반영되는지 확인
    - '오늘 선수별 점수계산' 시트에 선수 별 성적 입력
    


    
### 3. 누적 점수 계산 코드 실행
    - '트리오 예측 점수계산_(오늘날짜).xlsx' 파일과 '누적 점수 계산시트.xlsx' 파일 필요
    - '누적 점수 계산시트.xlsx'에 누적 점수 결과 저장
    - 매일 유저 별 / 선수 별 획득한 점수 기록 로그 저장
    
---

#### 1. 댓글 크롤러
---

In [2]:
# 필요 라이브러리 임포트 및 옵션 설정

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import time
import warnings
from datetime import datetime as dt
import re
warnings.filterwarnings('ignore')

# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

In [3]:
# 댓글 크롤링

options = webdriver.ChromeOptions()
options.add_argument('headless')

driver = webdriver.Chrome('./chromedriver', options=options)
driver.get('https://paige.kr.nc.com/clubhouse/post-link?postId=460319') # 트리오 예측 링크 입력
action = ActionChains(driver)
time.sleep(1)

# 더 보기 다 눌러놓기
while True:
    time.sleep(1)
    try:
        driver.find_element(By.CLASS_NAME, 'button-more').click()
    except:
        print('더 보기 없음') # 더 보기 버튼 없을 시 출력
        break
        
# 위에서부터 댓글 크롤링
nickname = driver.find_elements(By.CLASS_NAME, 'nickname') #닉네임
content = driver.find_elements(By.CLASS_NAME, 'comment-content') #댓글 내용

#게시글 작성자 이름 빼기 (클래스 이름이 같음)
nickname = nickname[1:]

# 닉네임과 댓글 갯수 비교하여 정상적으로 크롤링 되었는지 확인
print('닉네임 갯수 : {}'.format(len(nickname)))
print('댓글 갯수 : {}'.format(len(content)))

# 엑셀파일로 저장 및 선수
comment_df = pd.DataFrame(data=[n.text for n in nickname], columns=['닉네임'])
comment_df['댓글'] = [c.text for c in content]
comment_df['Pick_1'] = 0
comment_df['Pick_2'] = 0
comment_df['Pick_3'] = 0

# 댓글 split하여 1픽, 2픽, 3픽 선수로 나눠 정렬
index_count = 0
for c in comment_df['댓글']:
    if len(c.split('/')) == 3:
        comment_df['Pick_1'][index_count] = re.sub(r'\s', '', c.split('/')[0])
        comment_df['Pick_2'][index_count] = re.sub(r'\s', '', c.split('/')[1])
        comment_df['Pick_3'][index_count] = re.sub(r'\s', '', c.split('/')[2])
    index_count +=1

더 보기 없음
닉네임 갯수 : 267
댓글 갯수 : 267


In [42]:
raw_data = comment_df

In [74]:
comment_df = raw_data

In [ ]:
comment_df = comment_df[comment_df['Pick_1'] != 0]

In [70]:
comment_df = comment_df[comment_df['Pick_1'].map(lambda x: len(str(x)) < 10)]
# comment_df = comment_df[comment_df['Pick_2'].map(lambda x: len(str(x)) < 10)]
# comment_df = comment_df[comment_df['Pick_3'].map(lambda x: len(str(x)) < 10)]

In [80]:
comment_df[comment_df['닉네임'] == 'EAGLES1']

,닉네임,댓글,Pick_1,Pick_2,Pick_3
30,EAGLES1,이정후(51)/박병호(52)/나성범(47),이정후(51),박병호(52),나성범(47)


In [79]:
comment_df = comment_df.drop_duplicates(['닉네임'])

In [73]:
len(comment_df)

855

In [3]:
# 댓글 dataframe 엑셀로 저장
comment_df.to_excel('comment_{}.xlsx'.format(dt.today().strftime("%m%d")))

#### 2. 누적 점수 계산 및 로그 기록
---

In [4]:
# 당일 계산 시트 및 누적 점수 계산시트 임포트
score_df = pd.read_excel('트리오예측 점수계산.xlsx', sheet_name=[0,1], index_col=0)
accm_df = pd.read_excel('누적 점수 계산시트.xlsx', sheet_name=[0,1,2,3], index_col=0)

In [5]:
# 유저 점수 결측값 0으로 대체 (엔트리에 없거나 양식을 잘못 쓴 경우 0점처리)
score_df[0] = score_df[0].fillna(0)
score_df[1] = score_df[1].fillna(0)

In [6]:
# 유저 누적 계산

for i in score_df[0].index:
    try:
        if score_df[0]['닉네임'][i] in list(accm_df[0]['닉네임']):
            accm_df[0]['점수'][accm_df[0][accm_df[0]['닉네임'] == score_df[0]['닉네임'][i]].index] += score_df[0]['점수'][i]
        else:
            df = pd.DataFrame({'닉네임' : score_df[0]['닉네임'][i], '점수' : score_df[0]['점수'][i]}, index = [0])
            accm_df[0] = accm_df[0].append(df, ignore_index = True)
    except:
        pass

In [12]:
def user_accm(user_score_df, user_accm_df):
    for i in user_score_df.index:
        try:
            if user_score_df['닉네임'][i] in list(user_accm_df['닉네임']):
                user_accm_df['점수'][user_accm_df[user_accm_df['닉네임'] == user_score_df['닉네임'][i]].index] += user_score_df['점수'][i]
            else:
                df = pd.DataFrame({'닉네임' : user_score_df['닉네임'][i], '점수' : user_score_df['점수'][i]}, index = [0])
                user_accm_df = user_accm_df.append(df, ignore_index = True)
        except:
            pass
        
    return user_accm_df

In [7]:
# 선수 누적 계산

for i in score_df[1].index:
    try:
        if score_df[1]['선수 이름'][i] in list(accm_df[1]['선수 이름']):
            accm_df[1]['점수'][accm_df[1][accm_df[1]['선수 이름'] == score_df[1]['선수 이름'][i]].index] += score_df[1]['총 점수'][i]
        else:
            df = pd.DataFrame({'선수 이름' : score_df[1]['선수 이름'][i], '점수' : score_df[1]['총 점수'][i]}, index = [0])
            accm_df[1] = accm_df[1].append(df, ignore_index = True)
    except:
        pass

In [1]:
def player_accm(player_score_df, player_accm_df):
    for i in player_score_df.index:
        try:
            if player_score_df['선수 이름'][i] in list(player_accm_df['선수 이름']):
                player_accm_df['점수'][player_accm_df[player_accm_df['선수 이름'] == player_score_df['선수 이름'][i]].index] += player_score_df['총 점수'][i]
            else:
                df = pd.DataFrame({'선수 이름' : player_score_df['선수 이름'][i], '점수' : player_score_df['총 점수'][i]}, index = [0])
                player_accm_df = player_accm_df.append(df, ignore_index = True)
        except:
            pass
        
    return player_accm_df

In [8]:
# 유저 로그 기록

accm_df[2]['점수_{}'.format(dt.today().strftime("%m%d"))] = 0

for i in score_df[0].index:
    try:
        if score_df[0]['닉네임'][i] in list(accm_df[2]['닉네임']):
            accm_df[2]['점수_{}'.format(dt.today().strftime("%m%d"))][accm_df[2][accm_df[2]['닉네임'] == score_df[0]['닉네임'][i]].index] += score_df[0]['점수'][i]
        else:
            df = pd.DataFrame({'닉네임' : score_df[0]['닉네임'][i], '점수_{}'.format(dt.today().strftime("%m%d")) : score_df[0]['점수'][i]}, index = [0])
            accm_df[2] = accm_df[2].append(df, ignore_index = True)
    except:
        pass

In [12]:
def user_log(user_score_df, user_log_df):
    user_log_df['점수_{}'.format(dt.today().strftime("%m%d"))] = 0
    
    for i in user_score_df.index:
        try:
            if user_score_df['닉네임'][i] in list(user_log_df['닉네임']):
                user_log_df['점수_{}'.format(dt.today().strftime("%m%d"))][user_log_df[user_log_df['닉네임'] == user_score_df['닉네임'][i]].index] += user_score_df['점수'][i]
            else:
                df = pd.DataFrame({'닉네임' : user_score_df['닉네임'][i], '점수_{}'.format(dt.today().strftime("%m%d")) : user_score_df['점수'][i]}, index = [0])
                user_log_df = user_log_df.append(df, ignore_index = True)
        except:
            pass
        
    return user_log_df

In [9]:
# 선수 로그 기록

accm_df[3]['점수_{}'.format(dt.today().strftime("%m%d"))] = 0

for i in score_df[1].index:
    try:
        if score_df[1]['선수 이름'][i] in list(accm_df[3]['선수 이름']):
            accm_df[3]['점수_{}'.format(dt.today().strftime("%m%d"))][accm_df[3][accm_df[3]['선수 이름'] == score_df[1]['선수 이름'][i]].index] += score_df[1]['총 점수'][i]
        else:
            df = pd.DataFrame({'선수 이름' : score_df[1]['선수 이름'][i], '점수_{}'.format(dt.today().strftime("%m%d")) : score_df[1]['총 점수'][i]}, index = [0])
            accm_df[3] = accm_df[3].append(df, ignore_index = True)
    except:
        pass

In [15]:
def player_log(player_score_df, player_log_df):
    player_log_df['점수_{}'.format(dt.today().strftime("%m%d"))] = 0
    
    for i in player_score_df.index:
        try:
            if player_score_df['선수 이름'][i] in list(player_log_df['선수 이름']):
                player_log_df['점수_{}'.format(dt.today().strftime("%m%d"))][player_log_df[player_log_df['선수 이름'] == player_score_df['선수 이름'][i]].index] += player_score_df['총 점수'][i]
            else:
                df = pd.DataFrame({'선수 이름' : player_score_df['선수 이름'][i], '점수_{}'.format(dt.today().strftime("%m%d")) : player_score_df['총 점수'][i]}, index = [0])
                player_log_df = player_log_df.append(df, ignore_index = True)
        except:
            pass
        
    return player_log_df

In [17]:
test = player_log(score_df[1], accm_df[3])
test

,선수 이름,점수_0928,점수_0929,점수_1006
0,채은성,7.5,3.5,3.5
1,나성범,3.0,5.0,5.0
2,김현수,2.0,5.0,5.0
3,김혜성,3.0,5.0,5.0
4,소크라테스,5.0,6.0,6.0
5,오지환,5.5,7.5,7.5
6,이정후,6.0,9.5,9.5
7,이창진,3.5,3.5,3.5
8,푸이그,NaN,5.0,5.0
9,이재원,NaN,5.0,5.0


In [10]:
#1. 파일 생성
writer=pd.ExcelWriter('누적 점수 계산시트.xlsx', engine='openpyxl')
 
#2. 생성 파일에 시트명 지정 후 dataframe에 저장한 결과값 넣기
accm_df[0].to_excel(writer, sheet_name='유저 누적')
accm_df[1].to_excel(writer, sheet_name='선수 누적')
accm_df[2].to_excel(writer, sheet_name='유저 로그')
accm_df[3].to_excel(writer, sheet_name='선수 로그')
 
#3. 작성 완료 후 파일 저장
writer.save()
